# Use Vgg16 model with Keras

[State Farm Distracted Driver Detection | Kaggle](https://www.kaggle.com/c/state-farm-distracted-driver-detection/data)

In [ ]:
%pwd

In [72]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state/"

import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '../utils'))
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

In [88]:
DATA_HOME_DIR = "/home/ubuntu/datasets/state-farm-distracted-driver-detection/"

In [ ]:
%cd $DATA_HOME_DIR

In [ ]:
%cd train

In [ ]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
# create c0 .. c9 in train/valid directory
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [ ]:
from shutil import copyfile

In [ ]:
# copy c?/*.jpg to sample/train/c?/*.jpg
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
%cd ../valid

In [ ]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [ ]:
%cd ../../..

## train data with pre-trained model

In [62]:
import numpy as np
import pandas as pd
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

In [63]:
def ConvBlock(layers, model, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [64]:
def ConvBlock(layers, model, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))


def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

In [79]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

def VGG_16BN():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)
    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    return model


def finetune(model, num_classes):
    # remove last layer
    model.pop()
    # set all layers untrainable.
    for layer in model.layers: layer.trainable=False
    # add new layer
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def get_batches(path, dirname, gen=image.ImageDataGenerator(), shuffle=True,
                batch_size=64, class_mode='categorical'):
    gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
    return gen.flow_from_directory(path+dirname, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

get pre-trained weight from fast.ai server http://www.platform.ai/models/

```wget http://files.fast.ai/models/vgg16.h5```

You can also download [here](https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view)

In [69]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=nb_epoch, 
                        validation_data=val_batches, nb_val_samples=val_batches.N)

In [74]:
#model = VGG_16BN()
model = utils.vgg_ft_bn(1000)

# load pre-trained weights!!!
#model.load_weights('vgg16_bn.h5')

# remove last layer and add new layer
# ftmodel = finetune(model,10)

model.pop()
model.add(Dense(10, activation='softmax'))
layers = model.layers
opt = RMSprop(lr=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
for layer in layers[12:]: layer.trainable=True
K.set_value(opt.lr, 0.001)


model.summary()

553369600/553620808 [============================>.] - ETA: 0s____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_22 (Lambda)               (None, 3, 224, 224)   0           lambda_input_22[0][0]            
____________________________________________________________________________________________________
zeropadding2d_262 (ZeroPadding2D)(None, 3, 226, 226)   0           lambda_22[0][0]                  
____________________________________________________________________________________________________
convolution2d_261 (Convolution2D)(None, 64, 224, 224)  0           zeropadding2d_262[0][0]          
____________________________________________________________________________________________________
zeropadding2d_263 (ZeroPadding2D)(None, 64, 226, 226)  0           convolution2d_261[0][0]          
_____________________________

In [87]:
batch_size=64
# path = DATA_HOME_DIR
path = "/home/ubuntu/datasets/state-farm-distracted-driver-detection/sample/"

In [80]:
batches = get_batches(path,'train', batch_size=batch_size)
val_batches = get_batches(path,'valid', batch_size=batch_size)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [85]:
# train finetuned model(only last layer)
no_of_epochs=1

for epoch in range(no_of_epochs):
    print("Running epoch: %d" % epoch)
    model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
    latest_weights_filename = 'ft%d.h5' % epoch
    model.save_weights(latest_weights_filename)

Running epoch: 0
Epoch 1/1
1500/1500 [==============================] - 64s - loss: 3.5555 - acc: 0.3187 - val_loss: 2.0109 - val_acc: 0.4120


In [ ]:
model.load_weights("ft0.h5")

## test data

In [89]:
test_batches = get_batches(DATA_HOME_DIR, 'test', batch_size=2*batch_size, class_mode=None)
preds = model.predict_generator(test_batches, test_batches.nb_sample)

Found 79726 images belonging to 1 classes.


In [90]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)
preds = do_clip(preds,0.93)

In [91]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
submission = pd.DataFrame(preds, columns=classes)
submission.insert(0, 'img', [a[8:] for a in test_batches.filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.007778,0.007778,0.007778,0.023860,0.527727,0.007778,0.084808,0.007778,0.329806,0.021379
1,img_14887.jpg,0.133472,0.537916,0.199265,0.007778,0.007778,0.009557,0.076317,0.007778,0.007778,0.031945
2,img_62885.jpg,0.148337,0.061166,0.007778,0.042405,0.053649,0.628524,0.014313,0.017195,0.007778,0.031956
3,img_45125.jpg,0.054057,0.015053,0.301728,0.007778,0.584601,0.007778,0.009273,0.007778,0.007778,0.023790
4,img_22633.jpg,0.007778,0.007778,0.549735,0.007778,0.221140,0.007778,0.075547,0.007778,0.137820,0.007778


In [92]:
submission.to_csv("submission4.csv", index=False)

In [93]:
FileLink("submission4.csv")

/home/ubuntu/nbs/lesson4/submission4.csv